In [1]:
import pickle
%matplotlib inline
import sys, os
sys.path.append("/Users/guillaumelaunay/work/DVL/python3/pyproteinsExt/src")
sys.path.append("/Users/guillaumelaunay/work/DVL/python3/pyproteins/src")
%load_ext autoreload

## De serialize sequence data

In [2]:
#fileTag='NOX_annotation_20180619-165729'
fileTag='NOX_annotation_fullPFAM_20180625-111432'
latest='/Users/guillaumelaunay/work/projects/NOX/' + fileTag + '.pickle'
data = pickle.load( open(latest, "rb" ) )
print(len(data))

4915


## Import NCBI ontology

In [3]:
import pyproteinsExt.ontology
taxonTree = pyproteinsExt.ontology.Ontology(file='/Users/guillaumelaunay/work/databases/ontology/ncbitaxon.owl')

In [23]:
n = taxonTree.onto.search(iri='http://purl.obolibrary.org/obo/NCBITaxon_9606')
n[0].label
print(dir(n[0]))
n[0]_name

['__class__', '__classcell__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__label', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_equivalent_to', '_get_instance_possible_relations', '_get_is_instance_of', '_instance_equivalent_to_changed', '_instance_is_a_changed', '_name', '_set_is_instance_of', 'differents', 'generate_default_name', 'get_equivalent_to', 'get_inverse_properties', 'get_iri', 'get_name', 'get_properties', 'iri', 'is_a', 'is_instance_of', 'name', 'namespace', 'set_equivalent_to', 'set_iri', 'set_name', 'storid']


'NCBITaxon_9606'

In [24]:
import json
##### Shallow node version

lineagePool = []
domainPool = {} # keys are taxid, values are list of PFAM ID
validProteins = {}
for _id in data:
    if 'isNoEukaryota' not in data[_id]:
        print ('Cant find Taxon status ', _id, data[_id]['taxid'])
        continue
    if not data[_id]['isNoEukaryota']:
        continue
    
    taxid=data[_id]['taxid']
    n = taxonTree.onto.search(iri='http://purl.obolibrary.org/obo/NCBITaxon_' + taxid)[0]
    
    cLineage = [n.label[0]]
    for t in taxonTree._getLineage(n):
        if t.label: 
            cLineage.append(t.label[0])
        lineagePool.append(cLineage)
    
#    lineagePool.append([t.label[0] for t in taxonTree._getLineage(n) if t.label ])
    validProteins[_id] = data[_id]
    
nodes=[]
for row in lineagePool:
    #print(row)
    nodes += row
nodes = [ { 'cnt' : 0, 'name' : n} for n in list(set(nodes))]
for n in nodes:
    n['taxid'] = taxonTree.onto.search(label=n['name'])[0]._name.replace('NCBITaxon_','')
    for row in lineagePool:
        if n['name'] in row:
            n['cnt'] += 1

class Link(object):
    def __init__(self, a, b):
        self.target = a
        self.source = b
    def __hash__(self):
        return hash(self.source + self.target)
    def __eq__(self, other):
        return self.target == other.target and self.source == other.source
    def __repr__(self):
        return str(self.__dict__)
    
links = []
for row in lineagePool:
    for i, item in enumerate(row[:-1]):
        links.append( Link(item, row[i+1]) )
links = list(set(links) )


# Store node taxon names using their iri
#for n in nodes:
#    print(taxonTree.onto.search(label=n['name'])[0].iri, type(taxonTree.onto.search(label=n['name'])[0].iri))
nodeAdress = { taxonTree.onto.search(label=n['name'])[0].iri : n   for n in nodes }
#print (nodeAdress.keys())

# Add protein node link and domains
pNodes = []
for pEntry in validProteins:
    pValue = validProteins[pEntry]
    pNodes.append({ 'cnt' : 0, 'name' : pEntry, 'taxid': pValue['taxid'], 'pfam' : list( pValue['hmmr'].keys())})
    
for pNode in pNodes:
    iri = 'http://purl.obolibrary.org/obo/NCBITaxon_' + pNode['taxid']    
    source = nodeAdress[iri]['name']
    target = pNode['name']

    links.append(Link(target, source))

    #print(target, source)
    #print(links[-1])
    nodes.append(pNode)

    
dataOut = {'links' : [ d.__dict__ for d in links], 'nodes' : nodes}
with open('/Users/guillaumelaunay/work/projects/NOX/' + fileTag + '_tree.json', 'w') as f:
    json.dump(dataOut, f)

Cant find Taxon status  tr|G4YKD8|G4YKD8_PHYSP 1094619
Cant find Taxon status  tr|G4YM94|G4YM94_PHYSP 1094619
Cant find Taxon status  tr|G4ZHZ1|G4ZHZ1_PHYSP 1094619
tr|A0A136KU56|A0A136KU56_9CHLR Chloroflexi bacterium OLB14
{'target': 'tr|A0A136KU56|A0A136KU56_9CHLR', 'source': 'Chloroflexi bacterium OLB14'}
tr|A0A2N1WYC7|A0A2N1WYC7_9GAMM Gammaproteobacteria bacterium HGW-Gammaproteobacteria-5
{'target': 'tr|A0A2N1WYC7|A0A2N1WYC7_9GAMM', 'source': 'Gammaproteobacteria bacterium HGW-Gammaproteobacteria-5'}
tr|A0A0P6XZE1|A0A0P6XZE1_9CHLR Levilinea saccharolytica
{'target': 'tr|A0A0P6XZE1|A0A0P6XZE1_9CHLR', 'source': 'Levilinea saccharolytica'}
tr|A3TNH7|A3TNH7_9MICO Janibacter sp. HTCC2649
{'target': 'tr|A3TNH7|A3TNH7_9MICO', 'source': 'Janibacter sp. HTCC2649'}
tr|S0YK37|S0YK37_ECOLX Escherichia coli KTE38
{'target': 'tr|S0YK37|S0YK37_ECOLX', 'source': 'Escherichia coli KTE38'}
tr|X5M7H4|X5M7H4_9RHIZ Candidatus Phaeomarinobacter ectocarpi
{'target': 'tr|X5M7H4|X5M7H4_9RHIZ', 'source': '

In [ ]:
#def extractPfamID(node):
#print(data.keys())
print(list(data['tr|A0A136KU56|A0A136KU56_9CHLR']['hmmr'].keys()))

In [ ]:
fr l

In [25]:

def createNode(nodeName, nodePool, linkPool):
    pNode = None
    for n in nodePool:
        if n['name'] == nodeName:
            pNode = n
            break
    if not pNode:
        raise ValueError('oups')
    
    #print(pNode['name'])
    cNode = { 'name' : pNode['name'], 'cnt' : pNode['cnt'], 'pfam' : [], 'taxid' : pNode['taxid'] }
    if 'pfam' in pNode:
        cNode['pfam'] = pNode['pfam']
    
    #u = [ link.target for link in linkPool if link.source == cNode['name'] ]
    #print(u)
    # Get all node occurences as source
    # add corresponding node to children array 
    cNode['children'] = [createNode(link.target, nodePool, linkPool) for link in linkPool if link.source == cNode['name']]

    for n in cNode['children']:
        cNode['pfam'] += n['pfam']
    
    cNode['pfam'] = list(set(cNode['pfam']))
    
    return cNode
                

                
rootNode = None

asTree = createNode('root', nodes, links)

with open('/Users/guillaumelaunay/work/projects/NOX/' + fileTag + '_d3tree.json', 'w') as f:
    json.dump(asTree, f)

## Visualize JSON file with D3

/Users/guillaumelaunay/work/projects/NOX : node index.js
    
will serve *latest.json* to browser

In [10]:
#for n in nodes:
#    if 'pfam' in n:
#        print (n)
        
for l in links:
    if l.target == 'tr|A0A136KU56|A0A136KU56_9CHLR':
        print (l)